In [ ]:
import time
import random
import numpy as np
from tqdm.auto import tqdm
from keras.datasets import mnist
from matplotlib import pyplot as plt

In [ ]:
#Loading Training data and test data
objects = mnist
(train_img, train_lab), (test_imag, test_lab) = objects.load_data()

train_size,width, height = train_img.shape
X_test = test_imag
test_image = (X_test.reshape(X_test.shape[0], (X_test.shape[1]*X_test.shape[2])).T)/255

X_train = train_img
train_image =(X_train.reshape(X_train.shape[0], (X_train.shape[1]*X_train.shape[2])).T)/255

In [ ]:
#Initializing the weights and bias
def init_params(input, neuron, output):
    weight1 = np.random.rand(neuron, input) - 0.5
    bias1 = np.random.rand(neuron, 1) - 0.5
    weight2 = np.random.rand(output, neuron) - 0.5
    bias2 = np.random.rand(output, 1) - 0.5
    return weight1, bias1, weight2, bias2

#ReLU activation fucntion
def ReLU(Z):
    return np.maximum(Z, 0)

#sigmoid activation fucntion
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

#softmax fucntion for output layer
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

# forward porpagation    
def forward_prop(W1, b1, W2, b2, X, activation):

    Z1 = W1.dot(X) + b1
    if activation == "ReLU":
        A1 = ReLU(Z1)
    else:
        A1 = sigmoid(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

#Derivative of activation function
def ReLU_deriv(Z):
    return Z > 0

# one hot encoding method
def one_hot_encoding(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

#bacward propagation
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot_encoding(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / train_size * dZ2.dot(A1.T)
    db2 = 1 / train_size * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / train_size * dZ1.dot(X.T)
    db1 = 1 / train_size * np.sum(dZ1)
    return dW1, db1, dW2, db2

#updating weights and bias
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, l_rate):
    W1 = W1 - l_rate * dW1
    b1 = b1 - l_rate * db1    
    W2 = W2 - l_rate * dW2  
    b2 = b2 - l_rate * db2    
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def train_network(X, Y, l_rate, iterations, neuron, output, activation):
    W1, b1, W2, b2 = init_params(X.shape[0],neuron, output )
    print(f"Training Network with {activation} activation fuction ....")
    for i in tqdm(range(iterations+1)):
        
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X, activation)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, l_rate)
        if i % 25 == 0:
            predictions = get_predictions(A2)
            print(f'Iteration: {i},\tTraining Accuracy: {get_accuracy(predictions, Y)} ')
    return W1, b1, W2, b2

In [ ]:
learning_rate = 0.2
iterations = 600
h_layer1_neuron = 100
output_layer = 10

#Choose which activation function needs to be use.
activation  = "ReLU"
#activation = "sigmoid"
W1, b1, W2, b2 = train_network(train_image, train_lab, learning_rate, iterations,h_layer1_neuron,output_layer, activation)


In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X,activation)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(image, W1, b1, W2, b2):
    prediction = make_predictions(image, W1, b1, W2, b2)
    return prediction


In [ ]:
plt.figure("Prediction", (50,50))

for i in range(30):
    index = random.randint(0, test_image.shape[0])
    image = test_image[:, index, None]
    prediction = test_prediction(image, W1, b1, W2, b2)
    label = test_lab[index]
    image = image.reshape((28, 28)) * 255
    plt.subplot(10, 10, i+1)
    plt.gray()
    plt.imshow(image)
    plt.title(f'Predicted :{prediction}, True Label: {label}')

plt.show()
    

In [ ]:
test_predictions = make_predictions(test_image, W1, b1, W2, b2)
print(f'Test accuracy: {get_accuracy(test_predictions, test_lab)}')